# 1. Introduction

### FINDINGS And Questions

1. Total 15 weather stations.
2. Total 1449 unique buildings.
3. 4 types of meters. [0, 1, 2, 3]
4. Why std is so high in square_feet column?
5. We have 117 years of data. Is there potential outlier in years_built column?
6. Do I need to clean the datasets before merging them to train dataset?
7. What is the min/max meter reading, and does it make intuitive sense?

## 2. Library Imports

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')

## 3. Exploratory Analysis

In [2]:
train_df = pd.read_csv('./../ashrae-energy-prediction/train.csv')
building_df = pd.read_csv('./../ashrae-energy-prediction/building_metadata.csv')
weather_df = pd.read_csv('./../ashrae-energy-prediction/weather_train.csv')

In [3]:
print(train_df.shape)
print(building_df.shape)
print(weather_df.shape)

(20216100, 4)
(1449, 6)
(139773, 9)


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


### Reducing DataFrame Size for Faster Performace

In [ ]:
# Memory Statistics
train_df.memory_usage(deep=True) * 1e-6

In [ ]:
# Code to chage datatypes of columns
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['building_id'] = train_df['building_id'].astype("int32")
train_df['meter_reading'] = train_df['meter_reading'].astype("float32")
train_df['meter'] = train_df['meter'].astype("int16")

weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])

In [ ]:
# Check Change in size
train_df.memory_usage(deep=True) * 1e-6

In [ ]:
train_df.info()

### Merging the datasets using foreign keys

In [ ]:
train_df = train_df.merge(building_df, on='building_id', how='left')

In [ ]:
train_df = train_df.merge(weather_df, on=['site_id', 'timestamp'], how='left')

In [ ]:
train_df.head()

### a. Analysis of training dataframe

In [ ]:
# Display top 5 rows
train_df.head()

In [ ]:
# Display bottom 5 rows
train_df.tail()

In [ ]:
train_df.dtypes

In [ ]:
train_df.info(memory_usage="deep")

In [ ]:
# Why std in so high?
train_df.square_feet.describe()

In [ ]:
# We have 117 years of data
# Is there potential outlier
train_df.year_built.hist(bins=100)
plt.show()

In [ ]:
train_df.floor_count.describe()

In [ ]:
# There are null values in year_built column
train_df.year_built.isnull().sum()

In [ ]:
# There are lot of null values in floor_count column
train_df.floor_count.isnull().sum()

In [ ]:
train_df.square_feet.hist(bins=30)
plt.plot()

In [ ]:
# Do any catrgories need to be combined?
plt.figure(figsize=(10, 8))
sns.countplot(y='primary_use', data=train_df)
plt.show()

In [ ]:
# Will removing floor count affect my model?
plt.figure(figsize=(10, 8))
sns.countplot(y='floor_count', data=train_df)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(y='site_id', data=building_df)
plt.show()

## 4. Data Cleaning

## 5. Feature Engineering

## 6. Algorithm Selection

## 7. Model Training

## 8. Insights and Analysis